In [2]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained('best_model_bert_intensity')
model = BertForSequenceClassification.from_pretrained('best_model_bert_intensity',use_safetensors=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

emotion_labels = ["anger", "fear", "joy", "sadness", "surprise"]

text = "ayush is sad and slightly happy"

inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)

inputs = {key: value.to(device) for key, value in inputs.items()}

model.eval()

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

probabilities = torch.sigmoid(logits)

intensity_predictions = probabilities.squeeze().cpu().numpy()

for i, emotion in enumerate(emotion_labels):
    print(f"Predicted intensity for {emotion}: {intensity_predictions[i]:.2f}")


Predicted intensity for anger: 0.17
Predicted intensity for fear: 0.30
Predicted intensity for joy: 0.88
Predicted intensity for sadness: 0.62
Predicted intensity for surprise: 0.37


In [3]:
import streamlit as st
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np

# ------------------------------
# Load model and tokenizer
# ------------------------------
@st.cache_resource(show_spinner=False)
def load_model():
    tokenizer = BertTokenizer.from_pretrained('best_model_bert_intensity')
    model = BertForSequenceClassification.from_pretrained(
        'best_model_bert_intensity',
        use_safetensors=True
    )
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    return tokenizer, model, device

tokenizer, model, device = load_model()

# Emotion labels
emotion_labels = ["anger", "fear", "joy", "sadness", "surprise"]

# ------------------------------
# Streamlit UI
# ------------------------------
st.title("Emotion Intensity Prediction")
st.write("Enter text below to predict emotion intensities:")

user_input = st.text_area("Your Text:", height=100)

if st.button("Predict") and user_input.strip():
    # Tokenize
    inputs = tokenizer(
        user_input,
        return_tensors='pt',
        padding=True,
        truncation=True,
        max_length=512
    )
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Predict
    with torch.no_grad():
        outputs = model(**inputs)
        probabilities = torch.sigmoid(outputs.logits).squeeze().cpu().numpy()

    # Display results
    st.subheader("Predicted Intensities:")
    for i, emotion in enumerate(emotion_labels):
        st.write(f"**{emotion.capitalize()}:** {probabilities[i]:.2f}")
        st.progress(min(1.0, float(probabilities[i])))

    # High intensity alert
    if np.max(probabilities) > 0.8:
        top_emotion = emotion_labels[np.argmax(probabilities)]
        st.warning(f"⚠️ High emotional intensity detected for **{top_emotion}**!")


2025-09-05 21:39:55.271 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 21:39:55.395 
  command:

    streamlit run c:\Users\rachi\ml\tf_latest\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-09-05 21:39:55.396 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 21:39:55.396 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 21:39:55.397 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 21:39:55.397 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 21:39:55.399 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 21:39:55.399 Thread 'MainThread':